# Test run
We will do a test run on 250 images from the test set in order to calculate the metrics for the model.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import cv2
import imageio

from IPython.display import Image
from models.archs.kbnet_s_arch import KBNet_s
from metrics import mse, psnr

In [2]:
SOURCE_FOLDER = r'test_set\image'

First, import the model

In [5]:
model = KBNet_s(img_channel=1)
model = nn.DataParallel(model)
model.eval()

weights = torch.load("models\pretrained\gau_gray_25.pth")
model.load_state_dict(weights["net"])

<All keys matched successfully>

Import the images

In [3]:
images = []
for file in os.listdir(SOURCE_FOLDER):
    img = cv2.imread(os.path.join(SOURCE_FOLDER, file), 0)
    # convert to numpy array
    img = np.array(img)
    # normalize
    img = img / np.max(img)
    images.append(img)

Add noise to the images

In [4]:
noisy_images = []
for img in images:
    noisy_img = img + np.random.normal(0, 0.1, img.shape)
    noisy_images.append(noisy_img)

Denoise the images

In [6]:
denoised_images = []
for i in range(len(noisy_images)):
    # convert to torch tensor
    noisy_img = torch.from_numpy(noisy_images[i]).unsqueeze(0).unsqueeze(0).float()

    # run the model
    with torch.no_grad():
        denoised_img = model(noisy_img)

    # convert to numpy array
    denoised_img = denoised_img.squeeze().squeeze().numpy()
    denoised_images.append(denoised_img)

Finally, calculate the metrics

In [8]:
mse_values = []
for img, denoised_img in zip(images, denoised_images):
    mse_value = mse(img, denoised_img)
    mse_values.append(mse_value)

psnr_values = []
for img, denoised_img in zip(images, denoised_images):
    psnr_value = psnr(img, denoised_img)
    psnr_values.append(psnr_value)

print("MSE: {:.4f}".format(np.mean(mse_values)))
print("PSNR: {:.4f}".format(np.mean(psnr_values)))

MSE: 0.0004
PSNR: 82.2514
